In [2]:
import sys
from Load import *
from UE import *
from Satellite import *
from Oracle import *
import time
import os

def initial_assignment(UEss, Satss, oracle):
    C = UEss[0].coverage_info
    if oracle is None:
        # random assign
        for ueid in UEss:
            ue = UEss[ueid]
            possible_satellites = np.where(C[ueid, :, 0] == 1)[0]
            assert (len(possible_satellites) > 0)
            satid = random.choice(possible_satellites)
            ue.serving_satellite = Satss[satid]
            ue.serving_satellite_history.append(satid)
    else:
        for ueid in UEss:
            ue = UEss[ueid]
            satid = oracle.query_init_satellite(ueid)
            assert (C[ueid, satid, 0] == 1)
            ue.serving_satellite = Satss[satid]
            ue.serving_satellite_history.append(satid)


def monitor_timestamp(env):
    while True:
        if env.now % 100 == 0:
            print(f"Current simulation time {env.now}", file=sys.stderr)
        yield env.timeout(1)


if __name__ == "__main__":
    SOURCE_ALG = sys.argv[1]
    CANDIDATE_ALG = sys.argv[2]
    # configuration
    # When oracle_simulation is False, the oracle_assignment could be True/False
    # When oracle_simulation is True, the oracle_assignment must be True
    if oracle_simulation:
        assert (oracle_assignment)
    random.seed(10)

    # Loading scenarios
    beginning = time.time()
    UEs_template, satellites_template, C = load_scenario()
    DURATION = C.shape[2]
    env = simpy.Environment()

    assignment_oracle = None
    simulation_oracle = None
    if oracle_assignment:
        assignment_oracle = Oracle()
    if oracle_simulation:
        simulation_oracle = assignment_oracle

    UEs = {}
    for ue_template in UEs_template:
        ID = ue_template.ID
        if ID not in UEs:
            UEs[ID] = UE(
                identity=ID,
                position_x=ue_template.x,
                position_y=ue_template.y,
                coverage_info=C,
                oracle=simulation_oracle,
                env=env
            )
        else:
            print("ERROR")

    Satellites = {}
    for sat_template in satellites_template:
        ID = sat_template.ID
        if ID not in Satellites:
            Satellites[ID] = Satellite(
                identity=ID,
                position_x=sat_template.x,
                position_y=sat_template.y,
                height=sat_template.h,
                coverage_r=sat_template.r,
                velocity=sat_template.v,
                sind=sat_template.sind,
                cosd=sat_template.cosd,
                coverage_info=C,
                max_access_opportunity=max_access_opportunity,
                max_access_slots=max_access_slots,
                env=env,
                oracle=simulation_oracle,
            )
        else:
            print("ERROR")

    for ueid in UEs:
        UEs[ueid].satellites = Satellites

    for satid in Satellites:
        Satellites[satid].UEs = UEs
        Satellites[satid].satellites = Satellites

    print(f"Loading scenario in the simulation takes {time.time() - beginning}")
    # ========= UE, SAT objectives are ready =========
    initial_assignment(UEs, Satellites, assignment_oracle)

    env.process(monitor_timestamp(env))

    print('============= Experiment Log =============')
    env.run(until=DURATION)
    print('============= Experiment Ends =============')
    counter = allCounters(Satellites, UEs)
    counter.give_result()


Loading scenario in the simulation takes 0.07218527793884277
============= Experiment Log =============
[13] UE 6 sends Satellite 1 the message {"task": "MEASUREMENT_REPORT", "candidates": [4, 7, 2], "utility": [3310, 3234, 3062], "from": 6, "to": 1}
[13] Satellite 1 sends Satellite 4 the message {"task": "HANDOVER_REQUEST", "ueid": 6, "candidates": [4, 7, 2], "utility": [3310, 3234, 3062], "priority_load": [0, [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,

Current simulation time 0
Current simulation time 100
Current simulation time 200
Current simulation time 300
Current simulation time 400
Current simulation time 500
Current simulation time 600
Current simulation time 700
Current simulation time 800
Current simulation time 900
Current simulation time 1000
Current simulation time 1100
Current simulation time 1200
Current simulation time 1300
Current simulation time 1400


[1417] UE 12 sends Satellite 6 the message {"task": "RANDOM_ACCESS", "serving_length": 3122, "from": 12, "to": 6}
[1417] Satellite 6 sends UE 12 the message {"task": "RANDOM_ACCESS_RESPONSE", "from": 6, "to": 12}
[1417] Satellite 6 sends Satellite 0 the message {"task": "HANDOVER_SUCCESS", "ueid": 12, "priority_load": [1, [2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0

Current simulation time 1500
Current simulation time 1600
Current simulation time 1700
Current simulation time 1800
Current simulation time 1900
Current simulation time 2000
Current simulation time 2100
Current simulation time 2200
Current simulation time 2300
Current simulation time 2400
Current simulation time 2500
Current simulation time 2600
Current simulation time 2700
Current simulation time 2800
Current simulation time 2900
Current simulation time 3000


[3061] UE 17 sends Satellite 2 the message {"task": "MEASUREMENT_REPORT", "candidates": [10, 13, 8], "utility": [3804, 3388, 3330], "from": 17, "to": 2}
[3061] Satellite 2 sends Satellite 10 the message {"task": "HANDOVER_REQUEST", "ueid": 17, "candidates": [10, 13, 8], "utility": [3804, 3388, 3330], "priority_load": [0, [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,

...

Current simulation time 15100
Current simulation time 15200
Current simulation time 15300
Current simulation time 15400
Current simulation time 15500
Current simulation time 15600
Current simulation time 15700
Current simulation time 15800
Current simulation time 15900
Current simulation time 16000
Current simulation time 16100
Current simulation time 16200
Current simulation time 16300
Current simulation time 16400
Current simulation time 16500
Current simulation time 16600
Current simulation time 16700
Current simulation time 16800
Current simulation time 16900
Current simulation time 17000
Current simulation time 17100
Current simulation time 17200
Current simulation time 17300
Current simulation time 17400
Current simulation time 17500
Current simulation time 17600
Current simulation time 17700
Current simulation time 17800
Current simulation time 17900
Current simulation time 18000


[17274] UE 11 sends Satellite 26 the message {"task": "MEASUREMENT_REPORT", "candidates": [32, 34, 36], "utility": [4000, 4000, 4000], "from": 11, "to": 26}
[17274] Satellite 26 sends Satellite 32 the message {"task": "HANDOVER_REQUEST", "ueid": 11, "candidates": [32, 34, 36], "utility": [4000, 4000, 4000], "priority_load": [0, [5, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0

Current simulation time 18100
Current simulation time 18200
Current simulation time 18300
Current simulation time 18400
Current simulation time 18500
Current simulation time 18600
Current simulation time 18700
Current simulation time 18800
Current simulation time 18900
Current simulation time 19000
Current simulation time 19100
Current simulation time 19200
Current simulation time 19300
Current simulation time 19400
Current simulation time 19500
Current simulation time 19600


============= Experiment Ends =============


Current simulation time 19700
Current simulation time 19800
Current simulation time 19900


In [2]:
counter.give_result(100)

Total signalling: 1939
Total handover: 277
removed 398901 elements


In [2]:
a = [2,3,4]
b = [1,2,3]
for x, b in zip(a, b):
    print(x,b)

2 1
3 2
4 3
